In [1]:
import pandas as pd
import os

In [2]:
files = os.listdir("../cleaned")
print("Cleaned Files:\n",files)
files2 = os.listdir("../raw")
print("Raw Files:\n",files2)

Cleaned Files:
 ['EPL.csv', 'spainranking.csv', 'engranking.csv', 'Liga.csv', 'itranking.csv', 'model_data.csv', 'SerieA.csv']
Raw Files:
 ['.players_19.csv.icloud', '.players_16.csv.icloud', '.big_five_1995-2019.csv.icloud', '.players_18.csv.icloud', '.players_15.csv.icloud', 'team_stats.xlsx', 'teams_and_leagues.csv', '.players_17.csv.icloud', '.players_20.csv.icloud']


In [3]:
xl = pd.ExcelFile("../raw/team_stats.xlsx")
xl.sheet_names

['team_stats', 'epl', 'Sheet3']

In [4]:
liga = pd.read_excel("../raw/team_stats.xlsx",sheet_name="team_stats",header=0)
epl = pd.read_excel("../raw/team_stats.xlsx",sheet_name="epl",header=0)
serieA = pd.read_excel("../raw/team_stats.xlsx",sheet_name="Sheet3",header=0)

In [5]:
liga.rename({'Rank':'Ranking'},axis=1, inplace=True)
epl.rename({'Rank':'Ranking'},axis=1, inplace=True)
serieA.rename({'Rank':'Ranking'},axis=1, inplace=True)

In [6]:
transfers_it = pd.read_csv("../../transfers/cleaned/italy_joined.csv", dtype={'Ranking':"Int64"})
transfers_it["Country"] = 'Italy'
transfers_es = pd.read_csv("../../transfers/cleaned/spain_joined.csv",dtype={'Ranking':"Int64"})
transfers_es["Country"] = 'Spain'
transfers_en = pd.read_csv("../../transfers/cleaned/england_joined.csv",dtype={'Ranking':"Int64"})
transfers_en["Country"] = 'England'
transfer_df = pd.concat([transfers_it,transfers_es,transfers_en])
del transfer_df["Unnamed: 0"]
transfer_df.head()

,Year,Teams,Ranking,avg player value (EU),avg player wage,Transfer Spend (Euros),MA,Country
0,2019,Juventus,1,22287121.0,113636.0,201.150,194.28,Italy
1,2019,Inter Milan,2,14953666.0,53733.0,171.284,127.64,Italy
2,2019,SS Lazio,3,11658939.0,40455.0,41.078,36.29,Italy
3,2019,Atalanta,4,8712321.0,38393.0,51.130,49.78,Italy
4,2019,AS Roma,5,9323709.0,31516.0,102.690,110.81,Italy


In [7]:
transfer_df = transfer_df[["Year","Ranking","Transfer Spend (Euros)","MA","Country"]]
transfer_df.dropna(subset=["Ranking"],inplace=True)
transfer_df.head()

,Year,Ranking,Transfer Spend (Euros),MA,Country
0,2019,1,201.150,194.28,Italy
1,2019,2,171.284,127.64,Italy
2,2019,3,41.078,36.29,Italy
3,2019,4,51.130,49.78,Italy
4,2019,5,102.690,110.81,Italy


In [8]:
eng_df = pd.read_csv(f"../cleaned/{files[0]}",encoding="UTF-8")
final_eng = eng_df.merge(epl,on=['Ranking','Year'],how='outer')
final_eng = final_eng.sort_values(["Year","Ranking"]).reset_index(drop=True)
final_eng['Goals+'].iloc[0:20] = final_eng['G+'].iloc[0:20]
final_eng['Goals-'].iloc[0:20] = final_eng['G-'].iloc[0:20]
final_eng['Points_x'].iloc[0:20] = final_eng['Points_y'].iloc[0:20]
final_eng['Teams'].iloc[-20:] = final_eng['Team'].iloc[-20:]
final_eng['Points_x'].iloc[-20:] = final_eng['Points_y'].iloc[-20:]
final_eng['Goals+'].iloc[-20:] = final_eng['G+'].iloc[-20:]
final_eng['Goals-'].iloc[-20:] = final_eng['G-'].iloc[-20:]
final_eng["Country"] = 'England'
final_eng = final_eng.drop(columns=["Team","Goals","G+","G-","Points_y"]).rename({"Points_x":"Points"},axis=1)

In [9]:
es_df = pd.read_csv(f"../cleaned/{files[3]}",encoding="UTF-8")
final_es = es_df.merge(liga,on=['Ranking','Year'],how='outer')
final_es = final_es.sort_values(["Year","Ranking"]).reset_index(drop=True)
final_es['Goals+'].iloc[0:20] = final_es['G+'].iloc[0:20]
final_es['Goals-'].iloc[0:20] = final_es['G-'].iloc[0:20]
final_es['Teams'].iloc[-20:] = final_es['Team'].iloc[-20:]
final_es['Points_x'].iloc[-20:] = final_es['Points_y'].iloc[-20:]
final_es['Goals+'].iloc[-20:] = final_es['G+'].iloc[-20:]
final_es['Goals-'].iloc[-20:] = final_es['G-'].iloc[-20:]
final_es["Country"] = 'Spain'
final_es = final_es.drop(columns=["Team","Goals","G+","G-","Points_y"]).rename({"Points_x":"Points"},axis=1)

In [10]:
it_df = pd.read_csv(f"../cleaned/{files[6]}",encoding="UTF-8")
final_it = it_df.merge(serieA,on=['Ranking','Year'],how='outer')
final_it = final_it.sort_values(["Year","Ranking"]).reset_index(drop=True)
final_it['Goals+'].iloc[0:20] = final_it['G+'].iloc[0:20]
final_it['Goals-'].iloc[0:20] = final_it['G-'].iloc[0:20]
final_it['Points_x'].iloc[0:20] = final_it['Points_y'].iloc[0:20]
final_it['Teams'].iloc[-20:] = final_it['Team'].iloc[-20:]
final_it['Points_x'].iloc[-20:] = final_it['Points_y'].iloc[-20:]
final_it['Goals+'].iloc[-20:] = final_it['G+'].iloc[-20:]
final_it['Goals-'].iloc[-20:] = final_it['G-'].iloc[-20:]
final_it["Country"] = 'Italy'
final_it = final_it.drop(columns=["Team","Goals","G+","G-","Points_y"]).rename({"Points_x":"Points"},axis=1)

In [11]:
leagues_df = pd.concat([final_eng,final_it,final_es])
leagues_df = leagues_df.merge(transfer_df, on=["Year","Ranking","Country"],how='outer').drop_duplicates()

In [12]:
final_eng.to_csv('../cleaned_final/epl.csv')
final_es.to_csv('../cleaned_final/liga.csv')
final_it.to_csv('../cleaned_final/serieA.csv')
leagues_df.to_csv('../cleaned_final/leagues.csv')